# Proyek Analisis Data: Air-quality-dataset
- **Nama:** Izzuddin Ahmad Afif
- **Email:** izzuddinafif@gmail.com
- **ID Dicoding:** izzuddinafif

## Menentukan Pertanyaan Bisnis


## Menentukan Pertanyaan Bisnis

- Bagaimana tren perubahan konsentrasi PM2.5 di masing-masing stasiun selama periode tertentu, dan faktor apa yang memengaruhinya?
- Faktor lingkungan atau aktivitas manusia apa yang memiliki korelasi signifikan dengan perbedaan kualitas udara antar stasiun?


## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st  # Jika diperlukan untuk deployment
import os
import glob

# Atur gaya plot secara default
plt.style.use('default')


## Data Wrangling

### Gathering Data

In [ ]:
# Load semua file CSV dalam folder 'csv'
folder_path = 'csv'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Dictionary untuk menampung data tiap stasiun
station_dataframes = {}

# Memuat data dari setiap file CSV
for file in csv_files:
    base_name = os.path.basename(file)
    parts = base_name.split('_')
    if len(parts) > 3:
        station = parts[2]  # Nama stasiun
        df = pd.read_csv(file)
        station_dataframes[station] = df

# Tampilkan contoh data (hanya menampilkan 5 baris) dari satu stasiun sebagai sample
sample_station = list(station_dataframes.keys())[0] if station_dataframes else None
if sample_station:
    print(f"Data awal untuk stasiun {sample_station}:")
    display(station_dataframes[sample_station].head())
else:
    print("Tidak ditemukan file CSV dalam folder.")


**Insight:**
- Semua file CSV memiliki struktur yang seragam yang menunjukkan bahwa data dikumpulkan dengan format yang standar. Hal ini akan memudahkan langkah cleaning data dan analisis selanjutnya. 
- File CSV yang berbeda per stasiun yang kemudian dikelompokkan menggunakan dictionary memungkinkan analisis per stasiun secara terpisah maupun perbandingan antar stasiun. 
- Melihat kolom year, month, day, dan hour, kita bisa mengidentifikasi bahwa data merupakan data time series. Ini membuka peluang untuk analisis tren temporal (misalnya, tren harian, bulanan, atau tahunan).
- Struktur data yang sudah terpisah per stasiun memudahkan penerapan teknik analisis lanjutan seperti perbandingan kualitas udara antar stasiun, analisis geospasial, ataupun clustering berdasarkan pola-pola tertentu.

### Assessing Data

In [ ]:
def assess(df):
    results = {}
    
    # 1. Bentuk DataFrame
    results["shape"] = df.shape
    
    # 2. Missing Values
    results["missing_values"] = df.isnull().sum().to_dict()
    
    # 3. Nilai Invalid (cek apakah bisa dikonversi ke float)
    numeric_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    invalid_values = {}
    for col in numeric_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                if pd.notnull(val):
                    try:
                        float(val)
                    except Exception:
                        count += 1
        invalid_values[col] = count
    results["invalid_values"] = invalid_values
    
    # 4. Baris duplikat
    results["duplicate_rows"] = int(df.duplicated().sum())
    
    # 5. Nilai tidak akurat (nilai negatif untuk kolom yang seharusnya non-negatif)
    non_negative_cols = ["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "RAIN", "WSPM"]
    inaccurate_values = {}
    for col in non_negative_cols:
        count = 0
        if col in df.columns:
            for val in df[col]:
                try:
                    if float(val) < 0:
                        count += 1
                except Exception:
                    continue
        inaccurate_values[col] = count
    results["inaccurate_values"] = inaccurate_values
    
    # 6. Nilai tidak konsisten (untuk kolom non-numerik)
    inconsistent_values = {}
    for col in df.columns:
        if df[col].dtype == 'O':
            inconsistent_values[col] = df[col].dropna().unique().tolist()
    results["inconsistent_values"] = inconsistent_values
    
    # 7. Deteksi Outlier dengan metode IQR 
    outliers = {}
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if df[col].nunique() <= 1:
            outliers[col] = 0
            continue
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = sum(1 for x in df[col] if x < lower_bound or x > upper_bound)
    results["outliers"] = outliers
    
    return results

# Lakukan assessment untuk setiap stasiun dan kumpulkan ringkasan khusus untuk PM2.5
assessment_results = {}
summary_data = []

for station, df in station_dataframes.items():
    res = assess(df)
    assessment_results[station] = res
    summary_data.append({
       "Station": station,
       "Rows": res["shape"][0],
       "Missing_PM2.5": res["missing_values"].get("PM2.5", 0),
       "Invalid_PM2.5": res["invalid_values"].get("PM2.5", 0),
       "Duplicate_Rows": res["duplicate_rows"],
       "Outliers_PM2.5": res["outliers"].get("PM2.5", 0)
    })

summary_df = pd.DataFrame(summary_data)
print("Ringkasan Assessment untuk PM2.5 per stasiun:")
display(summary_df)

# Visualisasi sederhana: Missing values & Outlier PM2.5 per stasiun
plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Missing_PM2.5"], color="skyblue")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Missing PM2.5")
plt.title("Missing PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
plt.bar(summary_df["Station"], summary_df["Outliers_PM2.5"], color="salmon")
plt.xlabel("Stasiun")
plt.ylabel("Jumlah Outlier PM2.5")
plt.title("Outlier PM2.5 per Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Insight:**
- Semua file memiliki jumlah baris dan kolom yang sama (35.064 baris dan 18 kolom), yang menunjukkan bahwa data dikumpulkan dalam periode dan struktur yang konsisten antar stasiun.
- Jumlah nilai hilang pada kolom, terutama untuk PM2.5, bervariasi antar stasiun. Misalnya, ada stasiun yang memiliki missing value PM2.5 yang tinggi (seperti Shunyi dengan 913 nilai hilang) dibandingkan dengan stasiun lain (misalnya Wanliu dengan 382 nilai hilang). Ini mengindikasikan adanya variasi dalam kualitas pengumpulan data atau masalah pencatatan di masing-masing lokasi.
- Tidak ditemukan baris duplikat dan invalid pada data.
- Deteksi outlier menggunakan metode IQR menunjukkan adanya jumlah pencilan yang cukup tinggi pada kolom PM2.5 (misalnya 1626 outlier di Huairou).

### Cleaning Data

In [ ]:
def impute_outliers(s, lower_quantile=0.25, upper_quantile=0.75, k=1.5):
    Q1 = s.quantile(lower_quantile)
    Q3 = s.quantile(upper_quantile)
    IQR = Q3 - Q1
    lower_bound = Q1 - k * IQR
    upper_bound = Q3 + k * IQR
    median_val = s.median()
    
    s_imputed = s.copy()
    s_imputed[s < lower_bound] = median_val
    s_imputed[s > upper_bound] = median_val
    return s_imputed

# Proses cleaning dan imputasi untuk setiap stasiun
cleaned_dataframes = {}
stats_summary = []

for station, df in station_dataframes.items():
    # Gabungkan kolom tanggal menjadi kolom datetime
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
    # Pindahkan kolom datetime ke posisi kedua
    datetime_col = df.pop('datetime')
    df.insert(1, 'datetime', datetime_col)
    
    # Pastikan PM2.5 dalam bentuk numerik
    df['PM2.5'] = pd.to_numeric(df['PM2.5'], errors='coerce')
    
    # Imputasi missing value dengan interpolasi linier
    df.sort_values('datetime', inplace=True)
    df['PM2.5_clean'] = df['PM2.5'].interpolate(method='linear')
    
    # Imputasi outlier di PM2.5_clean dengan median
    df['PM2.5_imputed'] = impute_outliers(df['PM2.5_clean'])
    
    cleaned_dataframes[station] = df
    
    # Statistik ringkasan
    original_stats = df['PM2.5'].describe()
    clean_stats = df['PM2.5_clean'].describe()
    imputed_stats = df['PM2.5_imputed'].describe()
    
    stats_summary.append({
        "Station": station,
        "Original_Mean": original_stats.get('mean', np.nan),
        "Original_Median": original_stats.get('50%', np.nan),
        "Clean_Mean": clean_stats.get('mean', np.nan),
        "Clean_Median": clean_stats.get('50%', np.nan),
        "Imputed_Mean": imputed_stats.get('mean', np.nan),
        "Imputed_Median": imputed_stats.get('50%', np.nan)
    })

stats_df = pd.DataFrame(stats_summary)
print("Summary Statistik PM2.5 per Stasiun:")
display(stats_df)


**Insight:**
- Dengan menggabungkan kolom tanggal (year, month, day, hour) menjadi satu kolom datetime dan mengurutkan data berdasarkan waktu, kita mendapatkan deret waktu yang lebih konsisten. Hal ini memudahkan analisis tren PM2.5 secara time series.
- Penggunaan interpolasi linier untuk mengisi missing data membuat data PM2.5 menjadi lebih kontinu. Jika missing data tidak diisi, bisa terjadi celah yang mengganggu analisis tren. Dengan imputasi, distribusi data menjadi lebih lengkap.
- Dari statistik awal, nilai rata-rata (mean) PM2.5 cenderung lebih tinggi karena adanya nilai ekstrim. Setelah dilakukan imputasi outlier dengan mengganti nilai ekstrim menggunakan median, nilai mean menurun secara signifikan, sedangkan median tetap stabil. Ini menunjukkan bahwa outlier sebelumnya memberikan bias pada perhitungan rata-rata.
- Setelah proses cleaning, data PM2.5 menjadi lebih homogen dan siap untuk analisis tren serta perbandingan kualitas udara antar stasiun. Data yang sudah dibersihkan memberikan dasar yang lebih solid untuk menjawab pertanyaan bisnis, misalnya apakah terdapat tren penurunan atau kenaikan konsentrasi PM2.5 dan perbedaan antar stasiun.



## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
# --- EDA: Eksplorasi Data Umum ---

# 1. Tampilkan contoh data (misalnya 5 baris) dari setiap stasiun
print("Contoh data (5 baris) dari setiap stasiun setelah cleaning:")
for station, df in cleaned_dataframes.items():
    print(f"\nStasiun: {station}")
    display(df.head(5))

# 2. Ringkasan Statistik PM2.5 (Imputed) untuk semua stasiun
global_summary = []
for station, df in cleaned_dataframes.items():
    stats = df['PM2.5_imputed'].describe()
    stats['Station'] = station
    global_summary.append(stats)
summary_df2 = pd.DataFrame(global_summary)
print("Ringkasan Statistik PM2.5 (Imputed) per stasiun:")
display(summary_df2)

# 3. Agregasi Bulanan: Hitung rata-rata, nilai minimum, maksimum, dan std PM2.5 per stasiun per bulan
agg_list = []
for station, df in cleaned_dataframes.items():
    df['month'] = df['datetime'].dt.month
    monthly_summary = df.groupby('month')['PM2.5_imputed'].agg(['mean', 'min', 'max', 'std']).reset_index()
    monthly_summary['Station'] = station
    agg_list.append(monthly_summary)
agg_all = pd.concat(agg_list)
print("Agregasi Bulanan PM2.5 (Imputed) untuk semua stasiun:")
display(agg_all)

# 4. Visualisasi garis: Tren bulanan PM2.5 untuk setiap stasiun
pivot_monthly = agg_all.pivot(index='month', columns='Station', values='mean')
plt.figure(figsize=(10,6))
for station in pivot_monthly.columns:
    plt.plot(pivot_monthly.index, pivot_monthly[station], marker='o', label=station)
plt.xlabel("Bulan")
plt.ylabel("Rata-rata PM2.5 (Imputed)")
plt.title("Tren Bulanan Rata-rata PM2.5 per Stasiun")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


**Insight**

- **Tampilan Data Awal & Statistik Deskriptif:**  
  - Tampilan 5 baris pertama dari setiap stasiun memberikan gambaran awal mengenai struktur data yang konsisten setelah dilakukan cleaning.  
  - Ringkasan statistik untuk PM2.5 (setelah imputasi) menunjukkan perbedaan signifikan antar stasiun. Misalnya, jika nilai rata-rata jauh lebih tinggi dibandingkan dengan median, hal ini mengindikasikan adanya outlier ekstrim yang perlu mendapat perhatian lebih.

- **Agregasi Bulanan & Pola Musiman:**  
  - Proses agregasi bulanan (menghitung rata-rata, nilai minimum, maksimum, dan standar deviasi) mengungkap pola musiman yang kuat.  
  - Hasil agregasi menunjukkan bahwa sebagian besar stasiun mengalami peningkatan konsentrasi PM2.5 pada bulan-bulan tertentu (misalnya, musim dingin atau periode transisi) dan penurunan pada bulan dengan kondisi cuaca yang mendukung pengenceran polutan.

- **Visualisasi Tren Bulanan:**  
  - Plot garis tren bulanan rata-rata PM2.5 memberikan gambaran visual tentang fluktuasi dan perbedaan level antar stasiun.  
  - Meskipun pola naik-turun (trend) cenderung serupa, perbedaan nilai puncak dan dasar antara stasiun menandakan adanya faktor lokal (seperti kepadatan lalu lintas atau aktivitas industri) yang memengaruhi kualitas udara.

- **Validasi Konsistensi Data:**  
  - Gabungan tampilan data, statistik deskriptif, dan agregasi membantu memastikan bahwa data telah di-cleaning dengan baik dan siap untuk analisis lebih lanjut.  
  - Data agregasi yang disusun mendukung identifikasi pola yang mendalam, memberikan dasar yang kuat untuk visualisasi lanjutan serta interpretasi terkait pertanyaan bisnis.

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
# --- Pertanyaan 1: Tren Perubahan Konsentrasi PM2.5 ---

# Visualisasi: Plot garis tren bulanan rata-rata PM2.5 untuk setiap stasiun
plt.figure(figsize=(10,6))
for station in pivot_monthly.columns:
    plt.plot(pivot_monthly.index, pivot_monthly[station], marker='o', label=station)
plt.xlabel("Bulan")
plt.ylabel("Rata-rata PM2.5 (Imputed)")
plt.title("Tren Bulanan Rata-rata PM2.5 per Stasiun")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# Catatan: Tabel 'agg_all' merupakan hasil agregasi yang dapat dijadikan dasar analisa tren.


**Insight**

- **Pola Musiman yang Konsisten:**  
  Grafik tren bulanan menunjukkan bahwa hampir semua stasiun mengalami kenaikan konsentrasi PM2.5 pada bulan-bulan tertentu—misalnya, menjelang akhir tahun atau awal tahun berikutnya—dan penurunan pada bulan-bulan pertengahan tahun. Pola ini dapat dikaitkan dengan faktor musiman seperti perubahan suhu, kelembapan, dan curah hujan.

- **Perbedaan Level Antar Stasiun:**  
  Meskipun pola fluktuasi (naik-turun) cenderung serupa di antara stasiun, terdapat perbedaan signifikan dalam nilai puncak dan dasar. Sebagai contoh, stasiun *Aotizhongxin* mungkin secara konsisten menunjukkan level PM2.5 yang lebih tinggi dibandingkan stasiun seperti *Changping*, yang mencerminkan perbedaan kondisi lokal seperti volume lalu lintas atau aktivitas industri.

- **Amplitudo Fluktuasi:**  
  Rentang perbedaan antara nilai tertinggi dan terendah (amplitudo) berbeda-beda antar stasiun. Stasiun dengan amplitudo fluktuasi yang besar menunjukkan perubahan kualitas udara yang ekstrem antara musim “bersih” dan “kotor,” yang bisa disebabkan oleh kondisi geografis (misalnya, adanya lembah) atau intensitas aktivitas tertentu.

- **Indikasi Periode Kritis:**  
  Bulan-bulan dengan konsentrasi PM2.5 tertinggi dapat diidentifikasi sebagai periode kritis di setiap stasiun. Informasi ini berguna untuk menentukan waktu-waktu ketika intervensi, seperti pembatasan lalu lintas atau peningkatan pengawasan emisi, harus difokuskan.

- **Dasar Kebijakan dan Perencanaan:**  
  Tren bulanan yang terlihat dalam grafik memberikan dasar yang kuat bagi pembuat kebijakan untuk merancang strategi mitigasi polusi udara yang tepat, dengan menyesuaikan intervensi berdasarkan periode dengan konsentrasi polutan yang tinggi.


### Pertanyaan 2:

In [ ]:
# --- Pertanyaan 2: Faktor Lingkungan & Perbedaan Kualitas Udara Antar Stasiun ---

# 1. Boxplot perbandingan distribusi PM2.5 antar stasiun
all_data = []
for station, df in cleaned_dataframes.items():
    temp_df = df[['datetime', 'PM2.5_imputed']].copy()
    temp_df['Station'] = station
    all_data.append(temp_df)
all_data_df = pd.concat(all_data)
plt.figure(figsize=(12,6))
sns.boxplot(x='Station', y='PM2.5_imputed', data=all_data_df)
plt.xlabel("Stasiun")
plt.ylabel("PM2.5 (Imputed)")
plt.title("Perbandingan Distribusi PM2.5 Antar Stasiun")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Heatmap Korelasi: Gunakan stasiun representative (misal, stasiun pertama) untuk melihat korelasi parameter
rep_station = list(cleaned_dataframes.keys())[0] if cleaned_dataframes else None
if rep_station:
    numeric_cols = ["PM2.5_imputed", "PM10", "SO2", "NO2", "CO", "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]
    corr_matrix = cleaned_dataframes[rep_station][numeric_cols].corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f"Matriks Korelasi - {rep_station}")
    plt.tight_layout()
    plt.show()
else:
    print("Tidak ada stasiun untuk analisa korelasi.")


**Insight**

- **Boxplot Perbandingan PM2.5:**
  - **Perbedaan Median dan Rentang:**  
    Boxplot menunjukkan bahwa beberapa stasiun memiliki median PM2.5 yang lebih tinggi dibandingkan stasiun lainnya. Misalnya, stasiun *Gucheng* dan *Nongzhanguan* tampak memiliki nilai median yang relatif tinggi, sedangkan stasiun seperti *Changping* atau *Wanliu* cenderung lebih rendah.
  - **Outlier Ekstrem:**  
    Hampir semua stasiun menunjukkan adanya outlier dengan nilai sangat tinggi (misalnya, di atas 200 µg/m³). Hal ini menandakan bahwa terdapat episode polusi ekstrem yang terjadi secara sporadis, terutama di stasiun seperti *Huairou*, *Gucheng*, dan *Wanshouxigong*.
  - **Variasi Antar Stasiun:**  
    Perbedaan rentang nilai antar stasiun mengindikasikan bahwa faktor lokal—misalnya, aktivitas industri, kepadatan lalu lintas, atau kondisi geografis—berperan penting dalam menentukan kualitas udara di masing-masing lokasi.

- **Matriks Korelasi di Stasiun Aotizhongxin:**
  - **Korelasi dengan Parameter Polusi Lain (PM10, NO2, CO):**  
    PM2.5 menunjukkan korelasi positif yang cukup kuat dengan PM10, NO2, dan CO, menandakan bahwa sumber emisi yang menghasilkan partikel halus juga cenderung menghasilkan polutan lain dari aktivitas pembakaran, seperti kendaraan bermotor atau industri.
  - **Korelasi Negatif dengan Parameter Meteorologi (TEMP, WSPM, RAIN):**  
    Terdapat korelasi negatif antara PM2.5 dengan suhu, kecepatan angin, dan curah hujan. Kondisi cuaca seperti hujan dan angin kencang membantu menurunkan konsentrasi polutan dengan mempercepat proses pembersihan udara.
  - **Korelasi Negatif dengan O3:**  
    O3 (ozon) cenderung memiliki hubungan berlawanan dengan PM2.5 karena proses pembentukan dan penguraiannya berbeda. Kondisi atmosfer yang mendukung pembentukan ozon (misalnya, cerah dan minim partikel) biasanya tidak mendukung akumulasi PM2.5.

## Analisis Lanjutan (Opsional)

# Conclusion

1. **Variabilitas Kualitas Udara Antar Stasiun:**
   - **Perbedaan Statistik:**  
     Data deskriptif menunjukkan bahwa nilai rata-rata (mean) dan median PM2.5 bervariasi signifikan antar stasiun. Misalnya, beberapa stasiun menunjukkan rata-rata mendekati 150 µg/m³ dengan median yang jauh lebih rendah, mengindikasikan adanya outlier ekstrim. Sebaliknya, stasiun dengan nilai median lebih rendah (misalnya sekitar 70–80 µg/m³) menunjukkan distribusi yang lebih stabil. Variabilitas ini menunjukkan perbedaan mendasar dalam kondisi lingkungan dan potensi sumber polusi lokal.
     
2. **Pola Musiman dan Fluktuasi Konsentrasi:**
   - **Agregasi Bulanan:**  
     Hasil agregasi bulanan mengungkapkan pola musiman yang konsisten. Hampir semua stasiun menunjukkan kenaikan konsentrasi PM2.5 pada bulan-bulan tertentu—misalnya, puncak polusi di musim dingin atau periode transisi—yang kemudian diikuti oleh penurunan pada bulan-bulan dengan kondisi meteorologi lebih menguntungkan. Fluktuasi ekstrem (amplitudo besar antara nilai tertinggi dan terendah) menunjukkan adanya periode kritis yang memerlukan perhatian khusus.
     
3. **Hubungan Antar Parameter Polusi dan Pengaruh Faktor Meteorologi:**
   - **Korelasi Positif dan Negatif:**  
     Matriks korelasi, terutama pada stasiun perwakilan, mengungkapkan korelasi positif yang kuat antara PM2.5 dengan PM10, NO2, dan CO. Hal ini menunjukkan bahwa sumber polusi, seperti aktivitas transportasi dan industri, cenderung menghasilkan berbagai jenis polutan secara simultan. Di sisi lain, korelasi negatif dengan parameter meteorologi (suhu, kecepatan angin, curah hujan) menegaskan bahwa kondisi cuaca yang lebih baik dapat membantu mengurangi konsentrasi polutan di udara.
     
4. **Implikasi untuk Kebijakan dan Intervensi:**
   - **Identifikasi Periode Kritis:**  
     Dengan mengenali bulan-bulan di mana konsentrasi PM2.5 mencapai puncak, data ini dapat menjadi dasar penjadwalan intervensi, seperti pembatasan lalu lintas atau peningkatan pengawasan emisi, terutama di stasiun yang menunjukkan fluktuasi ekstrem.
   - **Pendekatan Lokal:**  
     Perbedaan yang signifikan antar stasiun mengindikasikan bahwa strategi pengendalian polusi perlu disesuaikan dengan kondisi lokal. Stasiun yang memiliki median dan rentang yang lebih tinggi harus mendapatkan fokus lebih dalam hal mitigasi, sedangkan stasiun dengan variabilitas rendah mungkin memerlukan pendekatan yang berbeda.
     
**Rekomendasi:**
- **Pengawasan Intensif pada Periode Puncak:**  
  Lakukan pengawasan dan intervensi lebih intensif pada bulan-bulan dengan konsentrasi tertinggi, berdasarkan pola musiman yang telah teridentifikasi.
- **Studi Lanjutan di Lokasi dengan Variabilitas Tinggi:**  
  Lakukan analisis mendalam di stasiun dengan outlier ekstrim dan amplitudo fluktuasi yang besar untuk mengidentifikasi sumber polusi spesifik, sehingga intervensi dapat lebih terfokus.
- **Integrasi Data Meteorologi:**  
  Menerapkan sistem peringatan dini yang mengintegrasikan parameter meteorologi untuk memprediksi dan merespons potensi lonjakan polusi secara lebih efektif.
- **Kebijakan Pengendalian Emisi:**  
  Sesuaikan kebijakan pengendalian emisi berdasarkan karakteristik lokal, dengan mempertimbangkan kombinasi faktor polusi dan kondisi cuaca yang berpengaruh.
